In [1]:
from synDataLoader import syn_mitbih, mixed_mitbih
from DataLoader import mitbih_train, mitbih_test
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from sklearn.utils import resample
from torch.utils import data

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import h5py
import numpy as np

import torch 
import torch.nn as nn

from TransCGAN_model import *

In [11]:
# filename=r'D:\Hanze_Master\Thesis\Data\Capture24_seq\X_30.h5'
# dataset_name='X_30'

# # Load the HDF5 file
# # Loading the data back from the HDF5 file
# with h5py.File('D:\Hanze_Master\Thesis\Data\Capture24_seq\X_30.h5', 'r') as f:
#     X = f[dataset_name][:]
#     print(X.shape)  # Check the shape of the loaded data

# labels = np.load(r'D:\Hanze_Master\Thesis\Data\Capture24_seq\Y_WillettsSpecific2018.npy')
# print(labels.shape)

In [ ]:
# unique_labels, counts = np.unique(labels, return_counts=True)
# print("Labels: ", unique_labels)
# print("Counts: ", counts)
# print("Amount of Labels is: ", len(unique_labels))

In [ ]:
# # Assume `labels` is a list of string labels
# label_encoder = LabelEncoder()
# Y = label_encoder.fit_transform(labels)  # Converts labels to integers

# # You can get the mapping like this:
# label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
# print("Label Mapping:", label_mapping)

In [ ]:
# # Split into training and testing datasets (e.g., 80% train, 20% test)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, Y, test_size=0.2, random_state=42, stratify=Y
# )

# # Verify the shapes
# print("Training data shape:", X_train.shape)
# print("Testing data shape:", X_test.shape)
# print("Training labels shape:", y_train.shape)
# print("Testing labels shape:", y_test.shape)

In [2]:
class DataProcessor:
    def __init__(self, h5_file=r'D:\Hanze_Master\Thesis\Data\Capture24_seq\X_30.h5' , dataset_name='X_30', label_file=r'D:\Hanze_Master\Thesis\Data\Capture24_seq\Y_WillettsSpecific2018.npy', test_size=0.2, random_state=42):
        self.h5_file = h5_file
        self.dataset_name = dataset_name
        self.label_file = label_file
        self.test_size = test_size
        self.random_state = random_state

    def load_and_process_data(self):
        # Load the HDF5 file
        with h5py.File(self.h5_file, 'r') as f:
            X = f[self.dataset_name][:]
            print(f"Loaded data shape: {X.shape}")

        # Load labels
        labels = np.load(self.label_file)
        print(f"Loaded labels shape: {labels.shape}")

        # Print unique labels and their counts
        unique_labels, counts = np.unique(labels, return_counts=True)
        print("Labels: ", unique_labels)
        print("Counts: ", counts)
        print("Amount of Labels is: ", len(unique_labels))

        # Encode labels
        label_encoder = LabelEncoder()
        Y = label_encoder.fit_transform(labels)
        label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
        print("Label Mapping:", label_mapping)

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(
            X, Y, test_size=self.test_size, random_state=self.random_state, stratify=Y
        )

        # Verify the shapes
        print("Training data shape:", X_train.shape)
        print("Testing data shape:", X_test.shape)
        print("Training labels shape:", y_train.shape)
        print("Testing labels shape:", y_test.shape)

        return X_train, X_test, y_train, y_test, label_mapping

In [ ]:
data_processor = DataProcessor()
X_train, X_test, y_train, y_test, label_mapping = data_processor.load_and_process_data()

In [4]:
class mitbih_train(Dataset):
    def __init__(self, X_train, y_train, n_samples=50000, oneD=False):
        # Convert to Pandas DataFrame for resampling
        df = pd.DataFrame(X_train.reshape(X_train.shape[0], -1))
        df['label'] = y_train

        # Resampling data for each class dynamically for classes 0 through 9
        resampled_data = []
        for label in range(10):
            class_data = df[df['label'] == label]
            if not class_data.empty:
                resampled_class_data = resample(class_data, n_samples=n_samples, random_state=123, replace=True)
                resampled_data.append(resampled_class_data)

        # Combine resampled data
        train_dataset = pd.concat(resampled_data)

        # Separate features and labels
        self.X_train = train_dataset.iloc[:, :-1].values
        self.y_train = train_dataset['label'].values

        # Reshape X_train back to original dimensions
        num_samples = X_train.shape[2] if len(X_train.shape) == 3 else 1
        num_channels = X_train.shape[1] if len(X_train.shape) == 3 else 1
        self.X_train = self.X_train.reshape(self.X_train.shape[0], num_channels, num_samples)

        if oneD:
            self.X_train = self.X_train.reshape(self.X_train.shape[0], 1, -1)  # Flatten samples within each channel
        else:
            self.X_train = self.X_train.reshape(self.X_train.shape[0], num_channels, 1, num_samples)

        print(f'X_train shape is {self.X_train.shape}')
        print(f'y_train shape is {self.y_train.shape}')
        for label in range(10):
            print(f'The dataset includes {len(train_dataset[train_dataset["label"] == label])} samples of class {label}')

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self, idx):
        return self.X_train[idx], self.y_train[idx]

In [ ]:
# Create an instance of mitbih_train
train_dataset = mitbih_train(X_train, y_train)

In [6]:
# Create a DataLoader for the training dataset
train_loader = data.DataLoader(train_dataset, batch_size=16, num_workers=16, shuffle=True)

In [28]:
# class mitbih_train(Dataset):
#     def __init__(self, filename='./X_30_train.csv.gz', n_samples=50000, oneD=False):
#         data_train = pd.read_csv(filename)
        
#         # making the class labels for our dataset
#         data_0 = data_train[data_train['label'] == 0]
#         data_1 = data_train[data_train['label'] == 1]
#         data_2 = data_train[data_train['label'] == 2]
#         data_3 = data_train[data_train['label'] == 3]
#         data_4 = data_train[data_train['label'] == 4]
#         data_5 = data_train[data_train['label'] == 5]
#         data_6 = data_train[data_train['label'] == 6]
#         data_7 = data_train[data_train['label'] == 7]
#         data_8 = data_train[data_train['label'] == 8]
#         data_9 = data_train[data_train['label'] == 9]
        
#         data_0_resample = resample(data_0, n_samples=n_samples, 
#                                     random_state=123, replace=True)
#         data_1_resample = resample(data_1, n_samples=n_samples, 
#                                     random_state=123, replace=True)
#         data_2_resample = resample(data_2, n_samples=n_samples, 
#                                     random_state=123, replace=True)
#         data_3_resample = resample(data_3, n_samples=n_samples, 
#                                     random_state=123, replace=True)
#         data_4_resample = resample(data_4, n_samples=n_samples, 
#                                     random_state=123, replace=True)
#         data_5_resample = resample(data_5, n_samples=n_samples, 
#                                     random_state=123, replace=True)
#         data_6_resample = resample(data_6, n_samples=n_samples, 
#                                     random_state=123, replace=True)
#         data_7_resample = resample(data_7, n_samples=n_samples, 
#                                     random_state=123, replace=True)
#         data_8_resample = resample(data_8, n_samples=n_samples, 
#                                     random_state=123, replace=True)
#         data_9_resample = resample(data_9, n_samples=n_samples, 
#                                     random_state=123, replace=True)
        
#         train_dataset = pd.concat((data_0_resample, data_1_resample, 
#                                     data_2_resample, data_3_resample,
#                                     data_4_resample, data_5_resample,
#                                     data_6_resample, data_7_resample,
#                                     data_8_resample, data_9_resample))
        
#         self.X_train = train_dataset.iloc[:, :-1].values
#         if oneD:
#             self.X_train = self.X_train.reshape(self.X_train.shape[0], 1, self.X_train.shape[1])
#         else:
#             self.X_train = self.X_train.reshape(self.X_train.shape[0], 1, 1, self.X_train.shape[1])
#         self.y_train = train_dataset['label'].values
    
#         print(f'X_train shape is {self.X_train.shape}')
#         print(f'y_train shape is {self.y_train.shape}')
#         print(f'The dataset including {len(data_0_resample)} class 0, {len(data_1_resample)} class 1, {len(data_2_resample)} class 2, {len(data_3_resample)} class 3, {len(data_4_resample)} class 4, {len(data_5_resample)} class 5, {len(data_6_resample)} class 6, {len(data_7_resample)} class 7, {len(data_8_resample)} class 8, {len(data_9_resample)} class 9')
                
        
#     def __len__(self):
#         return len(self.y_train)
    
#     def __getitem__(self, idx):
#         return self.X_train[idx], self.y_train[idx]

In [ ]:
#load dataset
train_set = mitbih_train()
train_loader = data.DataLoader(train_set, batch_size=16, num_workers=16, shuffle=True)

In [2]:
syn_ecg = syn_mitbih(n_samples=800, reshape=True)

data shape is (4000, 3, 150)
labels shape is (4000,)
The dataset including 800 class 0, 800 class 1, 800 class 2, 800 class 3, 800 class 4


(3, 150)

In [8]:
def function(n_samples = 20000, seq_len = 150, reshape = False, model_path=r'D:\Hanze_Master\Thesis\Syn_data_gen\tts-cgan\logs\mitbithCGAN_2025_01_10_13_43_49\Model\checkpoint'):
        
    gen_net = Generator(seq_len=seq_len, channels=3, num_classes=10, latent_dim=100, data_embed_dim=10, 
                label_embed_dim=10 ,depth=3, num_heads=5, 
                forward_drop_rate=0.5, attn_drop_rate=0.5)
    GAN_ckp = torch.load(model_path)
    print(GAN_ckp)
    return gen_net.load_state_dict(GAN_ckp['gen_state_dict'])

In [9]:
gnet = function()

{'epoch': 16, 'gen_model': 'my_gen', 'dis_model': 'my_dis', 'gen_state_dict': OrderedDict([('l1.weight', tensor([[ 0.1080,  0.0830, -0.0115,  ..., -0.0504, -0.0785,  0.1053],
        [-0.0943, -0.2926,  0.1990,  ..., -0.2806, -0.2917, -0.3154],
        [-0.0140,  0.0785, -0.1598,  ...,  0.1957,  0.1401,  0.1590],
        ...,
        [-0.0207,  0.0938, -0.1907,  ...,  0.0136,  0.2387,  0.1077],
        [ 0.0846, -0.1579,  0.2489,  ..., -0.2947, -0.1449, -0.1944],
        [-0.0739,  0.0960, -0.0937,  ..., -0.0706,  0.1110, -0.0488]],
       device='cuda:0')), ('l1.bias', tensor([-0.1248,  0.0073,  0.0890,  ..., -0.1020, -0.1791, -0.0924],
       device='cuda:0')), ('label_embedding.weight', tensor([[ 1.3118, -2.3489,  1.5202,  0.4000, -0.4130,  1.3274, -1.4441,  0.7752,
         -0.9089,  0.7938],
        [-1.2193, -1.4267, -1.2386, -0.1999,  0.9850, -0.9641, -0.1447,  1.5313,
         -0.6556,  1.0093],
        [ 0.8499,  1.0093, -0.9515, -0.5146, -0.0919,  0.8991,  0.0905, -0.7058,
  